# Lecture 8 - Root Finding: Bracketing Methods 🫚

## Comparing Bisection and False Position Methods

From C&C: Example 5.3-5.5

Recall that the velocity of a parachutist can be calculate analyticall using the following equation:

$$v(t) = \frac{gm}{c}(1 - e^{-ct/m})$$

Given the following known parameters: $m = 68.1$ kg, $t = 10$ s, and $v(t) = 40$ m/s

- Solve for $c$ using initial bracket $[12, 16]$
- Plot $log(|f(x)|)$ vs. number of iterations $n$

### 💪 Define the Function

In [ ]:
# Import Libraries
import numpy as np

# Define Parameters
m = 68.1
t = 10
v = 40
g = 9.81

# Adapt the equation above into a function, f(c), equals 0 when c reaches its true value.


### 💪 Create a Bisection Function

**Procedure of the Bisection Method**:
1. Choose $x_l$, $x_u$ such that $f(x_l) \cdot f(x_u) < 0$
2. Evaluate the midpoint: $x_r=(x_l + x_u)/2$
3. Update bracket according to:
  * If $f(x_l) \cdot f(x_r) < 0$: Set $x_u=x_r$
  * If $f(x_r) \cdot f(x_u) < 0$: Set $x_l=x_r$
4. Repeat until termination criteria is reached (e.g., while $|f(x)| > tol$, while $|x_r^{new}-x_r^{old}|>tol$).

In [ ]:
def bisection(f, xl, xu, tol=1e-8):

    # check for valid initial bracket
    assert f(xl) * f(xu) < 0, 'Choose xl,xu such that f(xl) * f(xu) < 0'
    err = 9999
    errors = [] # store errors to plot later

    while err > tol:
        xr = #[Fill in code here.]
        test = f(xl) * f(xr)
        if test < 0:
            #[Fill in code here.]
        elif test > 0:
            #[Fill in code here.]
        err = np.abs(f(xr))
        errors.append(err)

    return xr, errors

In [ ]:
# Run Bisection Method
xr, errors_b = bisection(f, 12, 16)

print(f'Estimate of root: {xr:0.4f}')
print(f'Iterations: {len(errors_b)}')

In [ ]:
# Compare to SciPy's Bisection Method Function
import scipy.optimize as sopt
sopt.bisect(f, 12, 16)

Read more about the SciPy's `sopt.bisect()` function [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html).

### 💪 Create a False Position Function

**Procedure of the False Position Method**:
1. Choose $x_l$, $x_u$ such that $f(x_l) \cdot f(x_u) < 0$
2. Evaluate the linear x-intercept: $x_r=x_u - \frac{f(x_u)(x_l-x_u)}{f(x_l)-f(x_u)}$
3. Update bracket according to:
  * If $f(x_l) \cdot f(x_r) < 0$: Set $x_u=x_r$
  * If $f(x_r) \cdot f(x_u) < 0$: Set $x_l=x_r$
4. Repeat until termination criteria is reached (e.g., while $|f(x)| > tol$, while $|x_r^{new}-x_r^{old}|>tol$).

Notice that false position method is the same as the bisection method, except for the $x_r$ update:

In [ ]:
# Create False Position Method Function and call it "falseposition()"
# Hint: It will be very similar to Bisection Method Function

In [ ]:
# Run False Position Method
xr, errors_f = falseposition(f, 12, 16)

print(f'Estimate of root: {xr:0.4f}')
print(f'Iterations: {len(errors_f)}')

# Note: SciPy doesn't have a built-in function for the False Position Method

### Comparison of Bisection and False Position Methods:

In [ ]:
import matplotlib.pyplot as plt
plt.semilogy(errors_b)
plt.semilogy(errors_f)
plt.ylabel('Log Error')
plt.xlabel('Number of iterations')
plt.legend(['Bisection', 'False Position'])
plt.show()

The false position method converges faster for this problem.

Note: These implementations are not very robust. They should include a `maxiter` to catch cases where the iteration does not converge. We could have chosen a different stopping criteria based on the difference in $x_r$ values rather than the $f(x_r)$ value.

## C&C: Case Study 8.2

Rain pH as a function of atmospheric CO2 concentration. Function defined below.

$$
[H^{+}]=\frac{K_1 K_H}{10^6 [H^{+}]}p_{CO_2}+2\frac{K_2 K_1 K_H}{10^6 [H^{+}]^2}p_{CO_2}+\frac{K_w}{[H^{+}]}
$$

Solve using bisection with initial bracket $[10^{-12}, 10^{-2}]$

In [ ]:
# Define Parameters
KH = 10**-1.46
K1 = 10**-6.3
K2 = 10**-10.3
Kw = 10**-14

# Partial pressure of CO2 (ppm)
# 1958: 315 ppm, 2003: 376 ppm, 2025: 428 ppm
pCO2 = 315

def f(H):
    return K1*KH*pCO2/(H*1e6) + 2*K2*K1*KH*pCO2/(H**2*1e6) + Kw/H - H

xr, errors_b = bisection(f, 1e-12, 1e-2)
pH = (-1*np.log10(xr))

print(f'Estimate of root: {xr:0.5e}')
print(f'Iterations: {len(errors_b)}')
print(f'pH = {pH:0.2f}')


❓ What happens to pH as $p_{CO_2}$ increases?
* Increasing $p_{CO_2}$ causes the pH to decrease (water becomes more acidic).

If we try false position instead, it will be very slow to converge.

In [ ]:
#xr, errors_f = falseposition(f, 1e-12, 1e-2)


❓ Why does the False Position Method struggle to converge?
* One side of our brackets is very close to 0 compared to the other, so the updated bracket value doesn't change significantly between iterations.
* The false position method assumes that the root is located near the side of the bracket with a function value closer to zero. However, that is not the case here: the root $~10^{-6}$ is much closer to $f(x_l)$. This will cause slow convergence for the false position method.

In [ ]:
# plot f(H) vs. H to visulaize where the root is.
x = np.linspace(1e-12, 1e-2, 100)
y = f(x)
plt.semilogx(x,y)
#plt.ylim([-1e-2,1e-2])
#plt.loglog(x,abs(y))
plt.xlabel('$[H^{+}]$')
plt.ylabel('$f([H^{+}])$')
plt.show()